In [ ]:
import argparse
import os, sys
import time
import datetime
from tqdm import tqdm_notebook as tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import numpy as np
from tools.dataset import CIFAR10
from torch.utils.data import Subset, DataLoader
import math

from PIL import Image, ImageFilter

In [ ]:
import torchvision.models as models

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class Resnet20(nn.Module):
  def __init__(self):
    super(Resnet20, self).__init__()
    self.conv1 = nn.Conv2d(3, 16, 3, stride=1, padding=1)
    self.bn1 = nn.BatchNorm2d(16)
    # 32 feature size layers
    self.conv2 = nn.Conv2d(16, 16, 3, stride=1, padding=1)
    self.bn2 = nn.BatchNorm2d(16)
    self.conv3 = nn.Conv2d(16, 16, 3, stride=1, padding=1)
    self.bn3 = nn.BatchNorm2d(16)
    self.shortcut3 = nn.Sequential()
    self.conv4 = nn.Conv2d(16, 16, 3, stride=1, padding=1)
    self.bn4 = nn.BatchNorm2d(16)
    self.conv5 = nn.Conv2d(16, 16, 3, stride=1, padding=1)
    self.bn5 = nn.BatchNorm2d(16)
    self.shortcut5 = nn.Sequential()
    self.conv6 = nn.Conv2d(16, 16, 3, stride=1, padding=1)
    self.bn6 = nn.BatchNorm2d(16)
    self.conv7 = nn.Conv2d(16, 16, 3, stride=1, padding=1)
    self.bn7 = nn.BatchNorm2d(16)
    self.shortcut7 = nn.Sequential()
    # 16 feature size layers
    self.conv8 = nn.Conv2d(16, 32, 3, stride=2, padding=1)
    self.bn8 = nn.BatchNorm2d(32)
    self.conv9 = nn.Conv2d(32, 32, 3, stride=1, padding=1)
    self.bn9 = nn.BatchNorm2d(32)
    # shortcut of mismatch sizes
    self.shortcut9 = nn.Sequential(
        nn.Conv2d(16, 32, 1, stride=2, bias=False),
        nn.BatchNorm2d(32)
    )
    self.conv10 = nn.Conv2d(32, 32, 3, stride=1, padding=1)
    self.bn10 = nn.BatchNorm2d(32)
    self.conv11 = nn.Conv2d(32, 32, 3, stride=1, padding=1)
    self.bn11 = nn.BatchNorm2d(32)
    self.shortcut11 = nn.Sequential()
    self.conv12 = nn.Conv2d(32, 32, 3, stride=1, padding=1)
    self.bn12 = nn.BatchNorm2d(32)
    self.conv13 = nn.Conv2d(32, 32, 3, stride=1, padding=1)
    self.bn13 = nn.BatchNorm2d(32)
    self.shortcut13 = nn.Sequential()
    # 8 feature size layers
    self.conv14 = nn.Conv2d(32, 64, 3, stride=2, padding=1)
    self.bn14 = nn.BatchNorm2d(64)
    self.conv15 = nn.Conv2d(64, 64, 3, stride=1, padding=1)
    self.bn15 = nn.BatchNorm2d(64)
    # shortcut of mismatch sizes
    self.shortcut15 = nn.Sequential(
        nn.Conv2d(32, 64, 1, stride=2, bias=False),
        nn.BatchNorm2d(64)
    )
    self.conv16 = nn.Conv2d(64, 64, 3, stride=1, padding=1)
    self.bn16 = nn.BatchNorm2d(64)
    self.conv17 = nn.Conv2d(64, 64, 3, stride=1, padding=1)
    self.bn17 = nn.BatchNorm2d(64)
    self.shortcut17 = nn.Sequential()
    self.conv18 = nn.Conv2d(64, 64, 3, stride=1, padding=1)
    self.bn18 = nn.BatchNorm2d(64)
    self.conv19 = nn.Conv2d(64, 64, 3, stride=1, padding=1)
    self.bn19 = nn.BatchNorm2d(64)
    self.shortcut19 = nn.Sequential()
    self.avgPool = nn.AvgPool2d((8, 8))
    self.fc = nn.Linear(64, 10)


  def forward(self, x):
    out = F.relu(self.bn1(self.conv1(x)))
    # start 32 feature size layer
    firstInput = out.clone()
    out = F.relu(self.bn2(self.conv2(out)))
    out = self.bn3(self.conv3(out))
    out += self.shortcut3(firstInput)
    out = F.relu(out)
    secondInput = out.clone()
    out = F.relu(self.bn4(self.conv4(out)))
    out = self.bn5(self.conv5(out))
    out += self.shortcut5(secondInput)
    out = F.relu(out)
    thirdInput = out.clone()
    out = F.relu(self.bn6(self.conv6(out)))
    out = self.bn7(self.conv7(out))
    out += self.shortcut7(thirdInput)
    out = F.relu(out)
    # start 16 feature size layer
    fourthInput = out.clone()
    out = F.relu(self.bn8(self.conv8(out)))
    out = self.bn9(self.conv9(out))
    out += self.shortcut9(fourthInput)
    out = F.relu(out)
    fifthInput = out.clone()
    out = F.relu(self.bn10(self.conv10(out)))
    out = self.bn11(self.conv11(out))
    out += self.shortcut11(fifthInput)
    out = F.relu(out)
    sixthInput = out.clone()
    out = F.relu(self.bn12(self.conv12(out)))
    out = self.bn13(self.conv13(out))
    out += self.shortcut13(sixthInput)
    out = F.relu(out)
    # start 8 feature size layer
    seventhInput = out.clone()
    out = F.relu(self.bn14(self.conv14(out)))
    out = self.bn15(self.conv15(out))
    out += self.shortcut15(seventhInput)
    out = F.relu(out)
    eighthInput = out.clone()
    out = F.relu(self.bn16(self.conv16(out)))
    out = self.bn17(self.conv17(out))
    out += self.shortcut17(eighthInput)
    out = F.relu(out)
    ninthInput = out.clone()
    out = F.relu(self.bn18(self.conv18(out)))
    out = self.bn19(self.conv19(out))
    out += self.shortcut19(ninthInput)
    out = F.relu(out)
    # global avg pool, fc, and softmax
    #out = self.avgPool(out)
    out = torch.flatten(out, 1)
    #out = self.fc(out) # removed due to details in the paper
    return out


In [ ]:
example_data = torch.randn(1,3,32,32)

net = Resnet20()
out = net.forward(example_data)
print(out.shape)

torch.Size([1, 4096])


In [ ]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [ ]:
def gaussian_blur(input_tensor, radius=1.0, device='cuda'):
    blurred_images = []

    for image_tensor in input_tensor:
        # Convert the image tensor to a PIL Image
        input_pil = transforms.ToPILImage()(image_tensor)

        # Apply Gaussian blur using the ImageFilter module
        blurred_pil = input_pil.filter(ImageFilter.GaussianBlur(radius=radius))

        # Convert the blurred PIL Image back to a tensor
        blurred_tensor = transforms.ToTensor()(blurred_pil)

        # Move the tensor to the specified device (GPU)
        blurred_tensor = blurred_tensor.to(device)

        blurred_images.append(blurred_tensor)

    return torch.stack(blurred_images)

In [ ]:
def train_encoder(net, projection_head, train_loader, base_lr, device, model_checkpoint, momentum=0.9, total_epochs=100):
    warmup_epochs = 10
    optimizer = optim.SGD(list(net.parameters()) + list(projection_head.parameters()), lr=base_lr, weight_decay=1e-6,
                               momentum=momentum)

    # optimizer = LARS(base_optimizer, trust_coef=1e-3)

    color_distort = get_color_distortion(s=1.0)


    color_augmentation = color_distort

    crop_flip_augmentation = transforms.Compose([
        transforms.RandomResizedCrop(size=(32,32), scale=(0.08, 1)),
        transforms.RandomHorizontalFlip()
    ])

    # GAUSSIAN BLUR
    # make & use method


    for epoch in range(total_epochs):
        net.train()
        projection_head.train()
        # Adjust learning rate
        cosine_decay_learning_rate(epoch, total_epochs, warmup_epochs, base_lr, optimizer)

        # Wrap the train_loader with tqdm for a progress bar
        total_loss = 0
        progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{total_epochs}", leave=False)
        for batch_idx, (inputs, _) in progress_bar:  # targets are not used in SimCLR
            inputs = inputs.to(device)
            net.to(device)
            projection_head.to(device)

            # Apply augmentations and forward pass
            augment_1 = color_augmentation(inputs)
            output_1 = net(augment_1)
            z1 = projection_head(output_1)

            # augment_1 = gaussian_blur(inputs)
            # output_1 = net(augment_1)
            # z1 = projection_head(output_1)

            # augment_1 = crop_flip_augmentation(inputs)
            # output_1 = net(augment_1)
            # z1 = projection_head(output_1)

            augment_2 = crop_flip_augmentation(inputs)
            output_2 = net(augment_2)
            z2 = projection_head(output_2)

            temperature = 0.5
            loss = nt_xent_loss(z1, z2, temperature)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            progress_bar.set_postfix(loss=total_loss / (batch_idx + 1))

        progress_bar.close()

        torch.save(net.state_dict(), model_checkpoint)

    return net

In [ ]:
def initialize_encoder_head():
  # initialize the base encoder
  net = Resnet20()

  input_size = 4096
  hidden_size = 2048
  output_size = 128

  # Initialize the nonlinear projection head
  projection_head = MLP(input_size, hidden_size, output_size)

  return net, projection_head

In [ ]:
def create_train_val(train_batch, val_batch):
  DATA_ROOT = "./data"
  TRAIN_BATCH_SIZE = train_batch
  VAL_BATCH_SIZE = val_batch

  transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
  ])

  transform_val = transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
  ])
  # construct dataset
  train_set = CIFAR10(
      root=DATA_ROOT,
      mode='train',
      download=True,
      transform=transform_train
  )
  val_set = CIFAR10(
      root=DATA_ROOT,
      mode='val',
      download=True,
      transform=transform_val
  )

  # construct dataloader
  train_loader = DataLoader(
      train_set,
      batch_size=TRAIN_BATCH_SIZE,
      shuffle=True,
      num_workers=4
  )

  val_loader = DataLoader(
      val_set,
      batch_size=VAL_BATCH_SIZE,
      shuffle=True,
      num_workers=4
  )

  return train_loader, val_loader,

In [ ]:
def create_train_finetune(train_batch):
  DATA_ROOT = "./data"
  TRAIN_BATCH_SIZE = train_batch
  transform_train = transforms.Compose([
      transforms.RandomCrop(32, padding=4),
      transforms.RandomHorizontalFlip(),
      transforms.ToTensor(),
      transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
  ])
  # construct dataset
  train_set = CIFAR10(
      root=DATA_ROOT,
      mode='train',
      download=True,
      transform=transform_train
  )
  # construct dataloader
  train_loader = DataLoader(
      train_set,
      batch_size=TRAIN_BATCH_SIZE,
      shuffle=True,
      num_workers=4
  )

  # # Get the length of the dataset
  # num_data = len(train_set)

  # # Use only 10% of the data
  # subset_indices = list(range(0, int(0.01 * num_data)))
  # subset_train_set = Subset(train_set, subset_indices)

  # # Construct the dataloader
  # train_loader = DataLoader(
  #     subset_train_set,
  #     batch_size=TRAIN_BATCH_SIZE,
  #     shuffle=True,
  #     num_workers=4
  # )

  return train_loader

In [ ]:
def create_test():
  DATA_ROOT = "./data"
  transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
  ])

  test_set = CIFAR10(
      root=DATA_ROOT,
      mode='test',
      download=True,
      transform=transform_test
  )
  # construct dataloader
  test_loader = DataLoader(
      test_set,
      batch_size=1,
      shuffle=True,
      num_workers=4
  )
  return test_loader

In [ ]:
def get_color_distortion(s=1.0):
# s is the strength of color distortion.
  color_jitter = transforms.ColorJitter(0.8*s, 0.8*s, 0.8*s, 0.2*s)
  rnd_color_jitter = transforms.RandomApply([color_jitter], p=0.8)
  rnd_gray = transforms.RandomGrayscale(p=0.2)
  color_distort = transforms.Compose([rnd_color_jitter,rnd_gray])
  return color_distort

In [ ]:
def nt_xent_loss(z1, z2, temperature=0.5):
    N = z1.size(0)  # batch size

    # Normalize the representations along the feature dimensions
    z1_norm = F.normalize(z1, dim=1)
    z2_norm = F.normalize(z2, dim=1)

    # Concatenate the normalized representations
    representations = torch.cat([z1_norm, z2_norm], dim=0)

    # Compute similarity matrix
    similarity_matrix = torch.matmul(representations, representations.T) / temperature

    # Mask to exclude self-contrast cases
    mask = torch.eye(2*N, dtype=torch.bool).to(z1.device)
    similarity_matrix = similarity_matrix.masked_fill_(mask, -1e9)

    # Labels: For each example in z1, its positive pair is in z2, and vice versa
    labels = torch.cat([torch.arange(N, 2*N), torch.arange(N)]).to(z1.device)

    # Calculate the NT-Xent loss
    loss = F.cross_entropy(similarity_matrix, labels)

    return loss

In [ ]:
def warmup_learning_rate(epoch, warmup_epochs, base_lr, optimizer):
    # Adjusts the learning rate for warmup phase """
    lr = base_lr * epoch / warmup_epochs
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

def cosine_decay_learning_rate(epoch, total_epochs, warmup_epochs, base_lr, optimizer):
    # Adjusts the learning rate following a cosine curve after warmup """
    if epoch < warmup_epochs:
        warmup_learning_rate(epoch, warmup_epochs, base_lr, optimizer)
    else:
        lr = 0.5 * base_lr * (1 + math.cos(math.pi * (epoch - warmup_epochs) / (total_epochs - warmup_epochs)))
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

In [ ]:
def encode_for_classifier(train_loader, device, net):
    net.eval()
    features = []
    labels = []

    with torch.no_grad():  # No need for gradient computation
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            inputs = inputs.to(device)
            output = net(inputs)
            features.append(output.cpu().detach())
            labels.append(targets.cpu().detach())
    X = torch.cat(features, dim=0).to(device)
    y = torch.cat(labels, dim=0).to(device)

    return X, y

In [ ]:
# from TA
class LogisticRegression(nn.Module):
    def __init__(self, n_features, n_classes):
        super(LogisticRegression, self).__init__()

        self.model = nn.Linear(n_features, n_classes)

    def forward(self, x):
        return self.model(x)

In [ ]:
def train_classifier_og(net, train_loader, val_loader, device, total_epochs):
    for param in net.parameters():
      param.requires_grad = False

    best_val_accuracy = 0

    # Prepare the data for L-BFGS optimization
    X, y = encode_for_classifier(train_loader, device, net)

    classifier = LogisticRegression(4096, 10).to(device)  # Adjust dimensions as needed
    classifier_optimizer = optim.LBFGS(classifier.parameters(), lr=1)

    for epoch in range(total_epochs):
        classifier.train()
        # Train Phase
        with tqdm(total=1, desc=f"Epoch {epoch+1}/{total_epochs} - Training", leave=False) as pbar_train:
            classifier_optimizer.zero_grad()
            outputs = classifier(X)
            loss = F.cross_entropy(outputs, y)
            loss.backward()
            classifier_optimizer.step()
            pbar_train.update(1)

        # Validation phase
        classifier.eval()
        correct, total = 0, 0
        with tqdm(val_loader, desc="Validating", leave=False) as pbar_val:
            for inputs, targets in pbar_val:
                inputs, targets = inputs.to(device), targets.to(device)
                features = net(inputs)
                outputs = classifier(features)
                _, predicted = torch.max(outputs.data, 1)
                total += targets.size(0)
                correct += (predicted == targets).sum().item()
                pbar_val.set_postfix(acc=(correct / total))

        val_accuracy = correct / total
        print(f'Epoch {epoch+1}/{total_epochs}: Validation Accuracy: {val_accuracy:.2f}')


In [ ]:
def train_classifier(net, train_loader, val_loader, device, total_epochs):
    for param in net.parameters():
        param.requires_grad = False

    best_val_accuracy = 0

    # Prepare the data for L-BFGS optimization
    X, y = encode_for_classifier(train_loader, device, net)

    labeled_indices = torch.randperm(len(y))
    labeled_indices = labeled_indices[:int(0.1 * len(y))]

    X_labeled = X[labeled_indices]
    y_labeled = y[labeled_indices]

    classifier = LogisticRegression(4096, 10).to(device)  # Adjust dimensions as needed
    classifier_optimizer = optim.LBFGS(classifier.parameters(), lr=1)

    for epoch in range(total_epochs):
        classifier.train()
        # Train Phase
        with tqdm(total=1, desc=f"Epoch {epoch+1}/{total_epochs} - Training", leave=False) as pbar_train:
            classifier_optimizer.zero_grad()
            outputs = classifier(X_labeled)
            loss = F.cross_entropy(outputs, y_labeled)
            loss.backward()
            classifier_optimizer.step()
            pbar_train.update(1)

        # Validation phase
        classifier.eval()
        correct, total = 0, 0
        with tqdm(val_loader, desc="Validating", leave=False) as pbar_val:
            for inputs, targets in pbar_val:
                inputs, targets = inputs.to(device), targets.to(device)
                features = net(inputs)
                outputs = classifier(features)
                _, predicted = torch.max(outputs.data, 1)
                total += targets.size(0)
                correct += (predicted == targets).sum().item()
                pbar_val.set_postfix(acc=(correct / total))

        val_accuracy = correct / total
        print(f'Epoch {epoch+1}/{total_epochs}: Validation Accuracy: {val_accuracy:.2f}')

    return classifier

In [ ]:
def fine_tune(net, train_loader, val_loader, device, total_epochs, model_checkpoint_net, model_checkpoint_clf):

  classifier = LogisticRegression(4096, 10).to(device)
  optimizer_clf = optim.SGD(list(classifier.parameters()), lr=0.005, weight_decay=1e-6,
                              momentum=0.9)

  optimizer_net = optim.SGD(list(net.parameters()), lr=0.01, weight_decay=1e-6,
                              momentum=0.9)

  best_val_acc = 0
  for epoch in range(total_epochs):
      net.train()
      classifier.train()
      # Adjust learning rate
      #cosine_decay_learning_rate(epoch, total_epochs, warmup_epochs, base_lr, optimizer)

      print("Epoch %d:", epoch)
      # this help you compute the training accuracy
      total_examples = 0
      correct_examples = 0

      total_loss = 0

      progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{total_epochs}", leave=False)
      for batch_idx, (inputs, targets) in progress_bar:  # targets are not used in SimCLR
          inputs = inputs.to(device)
          targets = targets.to(device)
          net.to(device)
          classifier.to(device)

          output_1 = net(inputs)
          output_2 = classifier(output_1)
          loss = F.cross_entropy(output_2, targets)


          total_loss += loss.item()

          # zero the gradient
          optimizer_clf.zero_grad()
          optimizer_net.zero_grad()
          # backpropagation
          loss.backward()

          # apply gradient and update the weights
          optimizer_clf.step()
          optimizer_net.step()
          # count the number of correctly predicted samples in the current batch

          predicted_classes = output_2.max(1)[1] # get most probable class for each predictions (max within column(class))
          total_examples += targets.size(0)
          equality_comparison = predicted_classes.eq(targets) # compare if each prediciton is correct
          correct_examples += equality_comparison.sum().item() # add trues from above to num of correct predictions (looked this up online)


          #total_loss += loss.item()
          progress_bar.set_postfix(loss=total_loss / (batch_idx + 1))

      progress_bar.close()

      avg_loss = total_loss / len(train_loader)
      avg_acc = correct_examples / total_examples
      print("Training loss: %.4f, Training accuracy: %.4f" %(avg_loss, avg_acc))

      # Validate on the validation dataset
      #######################
      # your code here
      # switch to eval mode
      net.eval()
      classifier.eval()

      #######################

      # this help you compute the validation accuracy
      total_examples = 0
      correct_examples = 0

      val_loss = 0 # again, track the validation loss if you want

          # disable gradient during validation, which can save GPU memory
      with torch.no_grad():
          for batch_idx, (inputs, targets) in enumerate(val_loader):
              ####################################
              # your code here
              # copy inputs to device
              inputs = inputs.to(device)
              targets = targets.to(device)

              # compute the output and loss
              output_1 = net(inputs)
              output_2 = classifier(output_1)
              loss = F.cross_entropy(output_2, targets)
              val_loss += loss.item()

              # count the number of correctly predicted samples in the current batch
              predicted_classes = output_2.max(1)[1] # get most probable class for each prediction
              total_examples += targets.size(0)
              equality_comparison = predicted_classes.eq(targets) # compare if each prediciton is correct
              correct_examples += equality_comparison.sum().item() # add trues from above to num of correct predictions

              ####################################

      avg_loss = val_loss / len(val_loader)
      avg_acc = correct_examples / total_examples
      print("Validation loss: %.4f, Validation accuracy: %.4f" % (avg_loss, avg_acc))

      # save the model checkpoint
      if avg_acc > best_val_acc:
          best_val_acc = avg_acc

          torch.save(net.state_dict(), model_checkpoint_net)
          torch.save(classifier.state_dict(), model_checkpoint_clf)

      print('')


  print(f"==> Optimization finished! Best validation accuracy: {best_val_acc:.4f}")


In [ ]:
def fine_tune_load(classifier, net, train_loader, val_loader, device, total_epochs, model_checkpoint_net, model_checkpoint_clf):
  optimizer_clf = optim.SGD(list(classifier.parameters()), lr=0.005, weight_decay=1e-6,
                              momentum=0.9)

  optimizer_net = optim.SGD(list(net.parameters()), lr=0.01, weight_decay=1e-6,
                              momentum=0.9)

  best_val_acc = 0
  for epoch in range(total_epochs):
      net.train()
      classifier.train()
      # Adjust learning rate
      #cosine_decay_learning_rate(epoch, total_epochs, warmup_epochs, base_lr, optimizer)

      print("Epoch %d:", epoch)
      # this help you compute the training accuracy
      total_examples = 0
      correct_examples = 0

      total_loss = 0

      progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{total_epochs}", leave=False)
      for batch_idx, (inputs, targets) in progress_bar:  # targets are not used in SimCLR
          inputs = inputs.to(device)
          targets = targets.to(device)
          net.to(device)
          classifier.to(device)

          output_1 = net(inputs)
          output_2 = classifier(output_1)
          loss = F.cross_entropy(output_2, targets)


          total_loss += loss.item()

          # zero the gradient
          optimizer_clf.zero_grad()
          optimizer_net.zero_grad()
          # backpropagation
          loss.backward()

          # apply gradient and update the weights
          optimizer_clf.step()
          optimizer_net.step()
          # count the number of correctly predicted samples in the current batch

          predicted_classes = output_2.max(1)[1] # get most probable class for each predictions (max within column(class))
          total_examples += targets.size(0)
          equality_comparison = predicted_classes.eq(targets) # compare if each prediciton is correct
          correct_examples += equality_comparison.sum().item() # add trues from above to num of correct predictions (looked this up online)


          #total_loss += loss.item()
          progress_bar.set_postfix(loss=total_loss / (batch_idx + 1))

      progress_bar.close()

      avg_loss = total_loss / len(train_loader)
      avg_acc = correct_examples / total_examples
      print("Training loss: %.4f, Training accuracy: %.4f" %(avg_loss, avg_acc))

      # Validate on the validation dataset
      #######################
      # your code here
      # switch to eval mode
      net.eval()
      classifier.eval()

      #######################

      # this help you compute the validation accuracy
      total_examples = 0
      correct_examples = 0

      val_loss = 0 # again, track the validation loss if you want

          # disable gradient during validation, which can save GPU memory
      with torch.no_grad():
          for batch_idx, (inputs, targets) in enumerate(val_loader):
              ####################################
              # your code here
              # copy inputs to device
              inputs = inputs.to(device)
              targets = targets.to(device)

              # compute the output and loss
              output_1 = net(inputs)
              output_2 = classifier(output_1)
              loss = F.cross_entropy(output_2, targets)
              val_loss += loss.item()

              # count the number of correctly predicted samples in the current batch
              predicted_classes = output_2.max(1)[1] # get most probable class for each prediction
              total_examples += targets.size(0)
              equality_comparison = predicted_classes.eq(targets) # compare if each prediciton is correct
              correct_examples += equality_comparison.sum().item() # add trues from above to num of correct predictions

              ####################################

      avg_loss = val_loss / len(val_loader)
      avg_acc = correct_examples / total_examples
      print("Validation loss: %.4f, Validation accuracy: %.4f" % (avg_loss, avg_acc))

      # save the model checkpoint
      if avg_acc > best_val_acc:
          best_val_acc = avg_acc

          torch.save(net.state_dict(), model_checkpoint_net)
          torch.save(classifier.state_dict(), model_checkpoint_clf)

      print('')


  print(f"==> Optimization finished! Best validation accuracy: {best_val_acc:.4f}")

In [ ]:
def test_classifier(net, classifier, test_loader, device):
    net.eval()  # Ensure the network is in evaluation mode
    classifier.eval()  # Ensure the classifier is in evaluation mode

    correct, total = 0, 0
    with torch.no_grad():  # No gradients needed
        for _ , (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)

            # Extract features and pass through the classifier
            features = net(inputs)
            outputs = classifier(features)
            _, predicted = torch.max(outputs.data, 1)

            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    test_accuracy = correct / total
    print(f'Test Accuracy: {test_accuracy:.2f}')

    return test_accuracy

In [ ]:
# starting the pipeline
# to generate the histogram: we are going to keep temperature at 0.5
# learning rates at (0.5, 1.0, 1.5)
# momentum will be kept at 0.9
# batch sizes can include (256, 512, 1024, 2048, 4096)

net, projection_head = initialize_encoder_head()
train_batch = 512 # CHANGE
val_batch = 100
base_lr = 1.0
momentum = 0.9
total_epochs = 10 # CHANGE
fine_tune_epochs = 10
model_checkpoint_encoder = 'encoder_b512_lr1p0_e10_e100'
model_checkpoint_net = 'net_b512_lr1p0_e10_e100'
model_checkpoint_clf = 'clf_b512_lr1p0_e10_e100'

train_loader, val_loader = create_train_val(train_batch, val_batch)
finetune_loader = create_train_finetune(train_batch)
net = train_encoder(net, projection_head, train_loader, base_lr, device, model_checkpoint_encoder, momentum, total_epochs)
fine_tune(net, finetune_loader, val_loader, device, fine_tune_epochs, model_checkpoint_net, model_checkpoint_clf)

Using downloaded and verified file: ./data/cifar10_trainval_F22.zip
Extracting ./data/cifar10_trainval_F22.zip to ./data
Files already downloaded and verified
Using downloaded and verified file: ./data/cifar10_trainval_F22.zip
Extracting ./data/cifar10_trainval_F22.zip to ./data
Files already downloaded and verified
Using downloaded and verified file: ./data/cifar10_trainval_F22.zip
Extracting ./data/cifar10_trainval_F22.zip to ./data
Files already downloaded and verified


<ipython-input-49-595b79b3349f>:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{total_epochs}", leave=False)


Epoch 1/10:   0%|          | 0/88 [00:00<?, ?it/s]

Epoch 2/10:   0%|          | 0/88 [00:00<?, ?it/s]

Epoch 3/10:   0%|          | 0/88 [00:00<?, ?it/s]

...

Training loss: 1.4069, Training accuracy: 0.4919
Validation loss: 1.3181, Validation accuracy: 0.5312

Epoch %d: 8


Epoch 9/10:   0%|          | 0/88 [00:00<?, ?it/s]

Training loss: 1.3768, Training accuracy: 0.5047
Validation loss: 1.2979, Validation accuracy: 0.5372

Epoch %d: 9


Epoch 10/10:   0%|          | 0/88 [00:00<?, ?it/s]

Training loss: 1.3571, Training accuracy: 0.5106
Validation loss: 1.2909, Validation accuracy: 0.5420

==> Optimization finished! Best validation accuracy: 0.5420


In [ ]:
torch.save(net.state_dict(), "net_b512_lr1p0_e50_e100")

In [ ]:
## IF ALREADY HAVE TRAINED ENCODER
net = Resnet20().to(device)
net.load_state_dict(torch.load("encoder_b512_lr1p0_e250_e100"))

train_batch = 512
val_batch = 250 # CHANGE
classify_total_epochs = 100

model_checkpoint_net = 'net_b512_lr1p0_e250_e100'
model_checkpoint_clf = 'clf_b512_lr1p0_e250_e100'

train_loader, val_loader = create_train_val(train_batch, val_batch)
finetune_loader = create_train_finetune(train_batch)

fine_tune(net, finetune_loader, val_loader, device, fine_tune_epochs, model_checkpoint_net, model_checkpoint_clf)

Using downloaded and verified file: ./data/cifar10_trainval_F22.zip
Extracting ./data/cifar10_trainval_F22.zip to ./data
Files already downloaded and verified
Using downloaded and verified file: ./data/cifar10_trainval_F22.zip
Extracting ./data/cifar10_trainval_F22.zip to ./data
Files already downloaded and verified
Using downloaded and verified file: ./data/cifar10_trainval_F22.zip
Extracting ./data/cifar10_trainval_F22.zip to ./data
Files already downloaded and verified
Epoch %d: 0


<ipython-input-44-6d5670df93ed>:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{total_epochs}", leave=False)


Epoch 1/100:   0%|          | 0/88 [00:00<?, ?it/s]

Training loss: 1.4865, Training accuracy: 0.5073
Validation loss: 1.1958, Validation accuracy: 0.5906

Epoch %d: 1


Epoch 2/100:   0%|          | 0/88 [00:00<?, ?it/s]

...

Training loss: 0.5163, Training accuracy: 0.8151
Validation loss: 0.6420, Validation accuracy: 0.7954

Epoch %d: 98


Epoch 99/100:   0%|          | 0/88 [00:00<?, ?it/s]

Training loss: 0.5127, Training accuracy: 0.8175
Validation loss: 0.6365, Validation accuracy: 0.7966

Epoch %d: 99


Epoch 100/100:   0%|          | 0/88 [00:00<?, ?it/s]

Training loss: 0.5081, Training accuracy: 0.8166
Validation loss: 0.6281, Validation accuracy: 0.7966

==> Optimization finished! Best validation accuracy: 0.7966


In [ ]:
torch.save(net.state_dict(), "net_b512_lr1p0_e250_e100")